In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

def get_data(trainFile, testFile, predFile, dropcols):
    # Load datasets
    df_train = pd.read_csv(trainFile)
    df_test = pd.read_csv(testFile)
    df_pred = pd.read_csv(predFile)

    # Drop specified columns
    df_train = df_train.drop(columns=dropcols)
    df_test = df_test.drop(columns=dropcols)

    ycol = 'Total Costs'
    Y_trn = None
    if ycol in df_train.columns:
        Y_trn = df_train[[ycol]]
        df_train = df_train.drop(columns=[ycol])

    X_trn_encoded = pd.get_dummies(df_train,columns=df_train.columns)
    X_tst_encoded = pd.get_dummies(df_test,columns=df_test.columns)

    return df_train, Y_trn, df_test, df_pred, X_trn_encoded, X_tst_encoded

# List of columns to remove
removeColumns = [
    'Operating Certificate Number', 'Zip Code - 3 digits', 'Gender', 'Race', 'CCSR Diagnosis Code',
    'CCSR Procedure Code', 'APR DRG Code', 'APR MDC Code', 'Payment Typology 2', 'Payment Typology 3',
    'Birth Weight', 'APR Severity of Illness Code'
]

removeColumns = ['Operating Certificate Number','Gender','Hospital Service Area','Emergency Department Indicator','Permanent Facility Id','Payment Typology 1',
 'Ethnicity','CCSR Diagnosis Description','Facility Name','Type of Admission','APR DRG Description','CCSR Procedure Description','Payment Typology 3',
 'Hospital County','CCSR Procedure Code','Race','Zip Code - 3 digits','APR MDC Code','Payment Typology 2','APR MDC Description','APR DRG Code','Birth Weight']

TrainPath = '/content/drive/MyDrive/COL774_AS1/train.csv'
TestPath = '/content/drive/MyDrive/COL774_AS1/test.csv'
PredPath = '/content/drive/MyDrive/COL774_AS1/test_pred.csv'
# Get the data
#X_train, Y_train, X_test, Y_test, X_train_encoded, X_test_encoded = get_data(TrainPath, TrainPath, PredPath, removeColumns)

print(TrainPath, TrainPath, PredPath, removeColumns)


/content/drive/MyDrive/COL774_AS1/train.csv /content/drive/MyDrive/COL774_AS1/train.csv /content/drive/MyDrive/COL774_AS1/test_pred.csv ['Operating Certificate Number', 'Gender', 'Hospital Service Area', 'Emergency Department Indicator', 'Permanent Facility Id', 'Payment Typology 1', 'Ethnicity', 'CCSR Diagnosis Description', 'Facility Name', 'Type of Admission', 'APR DRG Description', 'CCSR Procedure Description', 'Payment Typology 3', 'Hospital County', 'CCSR Procedure Code', 'Race', 'Zip Code - 3 digits', 'APR MDC Code', 'Payment Typology 2', 'APR MDC Description', 'APR DRG Code', 'Birth Weight']


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

def get_data(trainFile, testFile, predFile, dropcols):
    # Load datasets
    df_train = pd.read_csv(trainFile)
    df_test = pd.read_csv(testFile)
    df_pred = pd.read_csv(predFile)

    # Drop specified columns
    df_train = df_train.drop(columns=dropcols)
    df_test = df_test.drop(columns=dropcols)

    ycol = 'Total Costs'
    Y_trn = None
    if ycol in df_train.columns:
        Y_trn = df_train[[ycol]]
        df_train = df_train.drop(columns=[ycol])

    return df_train, Y_trn, df_test, df_pred

def one_hot_encode_and_scale(X_train, X_test, X_pred):
    # One-hot encode categorical variables
    X_train_encoded = pd.get_dummies(X_train)
    X_test_encoded = pd.get_dummies(X_test)
    X_pred_encoded = pd.get_dummies(X_pred)

    # Align columns of test and prediction data to training data
    X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)
    X_train_encoded, X_pred_encoded = X_train_encoded.align(X_pred_encoded, join='left', axis=1, fill_value=0)

    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_encoded)
    X_test_scaled = scaler.transform(X_test_encoded)
    X_pred_scaled = scaler.transform(X_pred_encoded)

    return X_train_scaled, X_test_scaled, X_pred_scaled, X_train_encoded.columns

def select_top_features_ridge(X_train, Y_train, X_test, X_pred, top_n=300):
    # Fit Ridge regression
    ridge = Ridge(alpha=1.0)
    ridge.fit(X_train, Y_train.values.ravel())

    # Get the absolute values of coefficients
    coefs = np.abs(ridge.coef_)

    # Select top N features
    top_features_idx = np.argsort(coefs)[-top_n:]
    X_train_top = X_train[:, top_features_idx]
    X_test_top = X_test[:, top_features_idx]
    X_pred_top = X_pred[:, top_features_idx]

    return X_train_top, X_test_top, X_pred_top

def calculate_mse_and_predict(X_train, Y_train, X_test, Y_test):
    # Train Ridge regression on selected features
    ridge = Ridge(alpha=1.0)
    ridge.fit(X_train, Y_train.values.ravel())

    # Predict on the test set
    Y_pred = ridge.predict(X_test)

    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(Y_test, Y_pred)
    return mse, Y_pred

# Define file paths
TrainPath = "train.csv"
TestPath = "test.csv"
PredPath = "pred.csv"

# List of columns to remove
removeColumns = [
    'Operating Certificate Number', 'Zip Code - 3 digits', 'Gender', 'Race', 'CCSR Diagnosis Code',
    'CCSR Procedure Code', 'APR DRG Code', 'APR MDC Code', 'Payment Typology 2', 'Payment Typology 3',
    'Birth Weight', 'APR Severity of Illness Code'
]

# Get the data
X_train, Y_train, X_test, X_pred = get_data(TrainPath, TestPath, PredPath, removeColumns)

# One-hot encode and scale the data
X_train_scaled, X_test_scaled, X_pred_scaled, feature_names = one_hot_encode_and_scale(X_train, X_test, X_pred)

# Select the best 300 features using Ridge regression
X_train_top, X_test_top, X_pred_top = select_top_features_ridge(X_train_scaled, Y_train, X_test_scaled, X_pred_scaled, top_n=300)

# Calculate MSE on the test set and get predictions
mse, Y_pred = calculate_mse_and_predict(X_train_top, Y_train, X_test_top, Y_train)

print("Mean Squared Error on the test set:", mse)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but SelectKBest was fitted without feature names
  warnings.warn(


Mean Squared Error: 14009555397386.83


In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('/content/drive/MyDrive/COL774_AS1/train.csv')

# Columns to be removed
Hospital Service Area,
Hospital County,
Operating Certificate Number,
Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Type of Admission,Patient Disposition,CCSR Diagnosis Code,CCSR Diagnosis Description,CCSR Procedure Code,CCSR Procedure Description,APR DRG Code,APR DRG Description,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Costs

removeColumns = [
    'Hospital Service Area',
    'Operating Certificate Number', 'Gender', , 'Emergency Department Indicator',
    'Permanent Facility Id', 'Payment Typology 1', 'Ethnicity', 'CCSR Diagnosis Description',
    'Facility Name', 'Type of Admission', 'APR DRG Description', 'CCSR Procedure Description',
    'Payment Typology 3', 'Hospital County', 'CCSR Procedure Code', 'Race', 'Zip Code - 3 digits',
    'APR MDC Code', 'Payment Typology 2', 'APR MDC Description', 'APR DRG Code', 'Birth Weight'
]

df1 = df.drop(columns=removeColumns)
categorical_columns = df1.columns
X_trn_encoded = pd.get_dummies(df1, columns=categorical_columns)
print(X_trn_encoded.shape)


In [ ]:
X_trn_encoded.head()

,Age Group,Patient Disposition,CCSR Diagnosis Code,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Total Costs
0,3,8,433,1,3.0,3.0,1,27153.03
1,5,19,29,3,2.0,2.0,1,13483.80
2,3,8,85,1,3.0,3.0,3,21851.50
3,2,8,112,2,4.0,3.0,3,19588.31
4,4,8,362,4,1.0,4.0,1,175220.97


In [ ]:
import pandas as pd
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import LabelEncoder

# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/COL774_AS1/train.csv')

df.columns

df['area'] = str(df['Facility Name']) + str(df['Zip Code - 3 digits'])
df['demographic'] = str(df['Age Group']) + str(df['Gender'])
df.drop(['Facility Name', 'Zip Code - 3 digits','Hospital Service Area', 'Hospital County','Operating Certificate Number', 'Permanent Facility Id'], axis=1, inplace=True)
df.columns

Index(['Age Group', 'Gender', 'Race', 'Ethnicity', 'Type of Admission',
       'Patient Disposition', 'CCSR Diagnosis Code',
       'CCSR Diagnosis Description', 'CCSR Procedure Code',
       'CCSR Procedure Description', 'APR DRG Code', 'APR DRG Description',
       'APR MDC Code', 'APR MDC Description', 'APR Severity of Illness Code',
       'APR Severity of Illness Description', 'APR Risk of Mortality',
       'APR Medical Surgical Description', 'Payment Typology 1',
       'Payment Typology 2', 'Payment Typology 3', 'Birth Weight',
       'Emergency Department Indicator', 'Total Costs', 'area'],
      dtype='object')

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/COL774_AS1/train.csv')

# Define the categorical columns to be target encoded
target_encode_cols = [
    'Type of Admission',
    'APR Severity of Illness Description',
    'APR Risk of Mortality'
]

# Function to perform target encoding with K-Fold
def target_encode(train, target, col, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mean_target = train[target].mean()
    encoded_col = pd.Series(0, index=train.index)

    for train_idx, valid_idx in kf.split(train):
        train_fold, valid_fold = train.iloc[train_idx], train.iloc[valid_idx]
        means = train_fold.groupby(col)[target].mean()
        encoded_col.iloc[valid_idx] = valid_fold[col].map(means)

    # Replace NaN values (if any) with the global mean
    encoded_col.fillna(mean_target, inplace=True)

    return encoded_col

# Apply target encoding to the selected columns
for col in target_encode_cols:
    df[col + '_encoded'] = target_encode(df, 'Total Costs', col)

# Drop the original columns if necessary
df = df.drop(columns=target_encode_cols)

# Display the first few rows of the encoded dataframe
print(df.head())


<ipython-input-3-59979d2a4a39>:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[29716.3860213  28249.53671448 28249.53671448 ... 29716.3860213
 29716.3860213  29716.3860213 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  encoded_col.iloc[valid_idx] = valid_fold[col].map(means)
<ipython-input-3-59979d2a4a39>:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[13426.90098346 13426.90098346 33513.14700224 ... 20511.96141274
 20511.96141274 72791.90073056]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  encoded_col.iloc[valid_idx] = valid_fold[col].map(means)
<ipython-input-3-59979d2a4a39>:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[15931.82895381 15931.82895381 26076.03971686 ... 1593

   Hospital Service Area  Hospital County  Operating Certificate Number  \
0                    6.0             22.0                     7001016.0   
1                    6.0              3.0                     7000006.0   
2                    6.0              3.0                     7000001.0   
3                    2.0             32.0                     3301007.0   
4                    6.0             26.0                     7002020.0   

   Permanent Facility Id  Facility Name  Age Group  Zip Code - 3 digits  \
0                 1301.0             70          3                 12.0   
1                 3058.0             93          5                  4.0   
2                 1178.0             11          3                  4.0   
3                  628.0            186          2                 32.0   
4                 1453.0             87          4                 14.0   

   Gender  Race  Ethnicity  ...  APR Medical Surgical Description  \
0       1     1          4  .

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/COL774_AS1/train.csv')

# Ensure the 'Total Costs' column is numeric
df['Total Costs'] = pd.to_numeric(df['Total Costs'], errors='coerce')

# Create the 'Hospital_Zip' feature
df['Hospital_Zip'] = df['Facility Name'].astype(str) + '_' + df['Zip Code - 3 digits'].astype(str)

def target_encode(train, target, col, alpha=1):
    mean_target = train[target].mean()  # Calculate the global mean of the target
    counts = train.groupby(col)[target].count()
    means = train.groupby(col)[target].mean()

    # Apply smoothing
    smoothed_means = (counts * means + alpha * mean_target) / (counts + alpha)

    return train[col].map(smoothed_means)

# Apply target encoding
df['Hospital_Zip_encoded'] = target_encode(df, 'Total Costs', 'Hospital_Zip')
df = df.drop(columns=['Hospital_Zip', 'Hospital Service Area', 'Hospital County', 'Operating Certificate Number', 'Permanent Facility Id', 'Facility Name', 'Zip Code - 3 digits', 'Payment Typology 2',
                      'Payment Typology 3', 'Birth Weight', 'CCSR Diagnosis Description', 'CCSR Procedure Description', 'APR DRG Description', 'APR MDC Description', 'APR Severity of Illness Code'])
df['Age Group'] = target_encode(df, 'Total Costs', 'Age Group')
df['Gender'] = target_encode(df, 'Total Costs', 'Gender')
df['Race'] = target_encode(df, 'Total Costs', 'Race')
df['Ethnicity'] = target_encode(df, 'Total Costs', 'Ethnicity')

df['CCSR Diagnosis Code'] = target_encode(df, 'Total Costs', 'CCSR Diagnosis Code')
df['CCSR Procedure Code'] = target_encode(df, 'Total Costs', 'CCSR Procedure Code')
df['APR DRG Code'] = target_encode(df, 'Total Costs', 'APR DRG Code')
df['APR MDC Code'] = target_encode(df, 'Total Costs', 'APR MDC Code')

# Define new label encoding mappings
severity_mapping = {3: 1, 4:2, 2:3, 1:4}
mortality_mapping = {3: 1, 4:2, 2:3, 1:4}
medical_surgical_mapping = {2: 0, 1: 1, 3: 2}
emergency_mapping = {1: 0, 2: 1}

df['APR Severity of Illness Description'] = df['APR Severity of Illness Description'].map(severity_mapping)
df['APR Risk of Mortality'] = df['APR Risk of Mortality'].map(mortality_mapping)
df['APR Medical Surgical Description'] = df['APR Medical Surgical Description'].map(medical_surgical_mapping)
df['Emergency Department Indicator'] = df['Emergency Department Indicator'].map(emergency_mapping)

df_encoded = pd.get_dummies(df, columns=['Patient Disposition', 'Payment Typology 1'])

# Define features (X) and target variable (y)
X_train = df_encoded.drop(columns=['Total Costs'])
y_train = df_encoded['Total Costs']

test_file = '/content/drive/MyDrive/COL774_AS1/test.csv'

X_train.shape

(584004, 42)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Load the training dataset
df = pd.read_csv('/content/drive/MyDrive/COL774_AS1/train.csv')

# Ensure the 'Total Costs' column is numeric
df['Total Costs'] = pd.to_numeric(df['Total Costs'], errors='coerce')

# Create the 'Hospital_Zip' feature
df['Hospital_Zip'] = df['Facility Name'].astype(str) + '_' + df['Zip Code - 3 digits'].astype(str)

def target_encode(train, target, col, alpha=1):
    mean_target = train[target].mean()  # Calculate the global mean of the target
    counts = train.groupby(col)[target].count()
    means = train.groupby(col)[target].mean()

    # Apply smoothing
    smoothed_means = (counts * means + alpha * mean_target) / (counts + alpha)

    return train[col].map(smoothed_means)

# Apply target encoding
df['Hospital_Zip_encoded'] = target_encode(df, 'Total Costs', 'Hospital_Zip')
df = df.drop(columns=['Hospital_Zip', 'Hospital Service Area', 'Hospital County', 'Operating Certificate Number', 'Permanent Facility Id', 'Facility Name', 'Zip Code - 3 digits', 'Payment Typology 2',
                      'Payment Typology 3', 'Birth Weight', 'CCSR Diagnosis Description', 'CCSR Procedure Description', 'APR DRG Description', 'APR MDC Description', 'APR Severity of Illness Code'])
df['Age Group'] = target_encode(df, 'Total Costs', 'Age Group')
df['Gender'] = target_encode(df, 'Total Costs', 'Gender')
df['Race'] = target_encode(df, 'Total Costs', 'Race')
df['Ethnicity'] = target_encode(df, 'Total Costs', 'Ethnicity')

df['CCSR Diagnosis Code'] = target_encode(df, 'Total Costs', 'CCSR Diagnosis Code')
df['CCSR Procedure Code'] = target_encode(df, 'Total Costs', 'CCSR Procedure Code')
df['APR DRG Code'] = target_encode(df, 'Total Costs', 'APR DRG Code')
df['APR MDC Code'] = target_encode(df, 'Total Costs', 'APR MDC Code')

# Define new label encoding mappings
severity_mapping = {3: 1, 4:2, 2:3, 1:4}
mortality_mapping = {3: 1, 4:2, 2:3, 1:4}
medical_surgical_mapping = {2: 0, 1: 1, 3: 2}
emergency_mapping = {1: 0, 2: 1}

df['APR Severity of Illness Description'] = df['APR Severity of Illness Description'].map(severity_mapping)
df['APR Risk of Mortality'] = df['APR Risk of Mortality'].map(mortality_mapping)
df['APR Medical Surgical Description'] = df['APR Medical Surgical Description'].map(medical_surgical_mapping)
df['Emergency Department Indicator'] = df['Emergency Department Indicator'].map(emergency_mapping)

df_encoded = pd.get_dummies(df, columns=['Patient Disposition', 'Payment Typology 1'])

# Define features (X) and target variable (y)
X_train = df_encoded.drop(columns=['Total Costs'])
y_train = df_encoded['Total Costs']

# Load the test dataset
df_test = pd.read_csv('/content/drive/MyDrive/COL774_AS1/test.csv')

# Apply the same preprocessing to the test data
df_test['Hospital_Zip'] = df_test['Facility Name'].astype(str) + '_' + df_test['Zip Code - 3 digits'].astype(str)
df_test['Hospital_Zip_encoded'] = target_encode(df, 'Total Costs', 'Hospital_Zip')

df_test = df_test.drop(columns=['Hospital_Zip', 'Hospital Service Area', 'Hospital County', 'Operating Certificate Number', 'Permanent Facility Id', 'Facility Name', 'Zip Code - 3 digits', 'Payment Typology 2',
                                'Payment Typology 3', 'Birth Weight', 'CCSR Diagnosis Description', 'CCSR Procedure Description', 'APR DRG Description', 'APR MDC Description', 'APR Severity of Illness Code'])
df_test['Age Group'] = target_encode(df, 'Total Costs', 'Age Group')
df_test['Gender'] = target_encode(df, 'Total Costs', 'Gender')
df_test['Race'] = target_encode(df, 'Total Costs', 'Race')
df_test['Ethnicity'] = target_encode(df, 'Total Costs', 'Ethnicity')

df_test['CCSR Diagnosis Code'] = target_encode(df, 'Total Costs', 'CCSR Diagnosis Code')
df_test['CCSR Procedure Code'] = target_encode(df, 'Total Costs', 'CCSR Procedure Code')
df_test['APR DRG Code'] = target_encode(df, 'Total Costs', 'APR DRG Code')
df_test['APR MDC Code'] = target_encode(df, 'Total Costs', 'APR MDC Code')

df_test['APR Severity of Illness Description'] = df_test['APR Severity of Illness Description'].map(severity_mapping)
df_test['APR Risk of Mortality'] = df_test['APR Risk of Mortality'].map(mortality_mapping)
df_test['APR Medical Surgical Description'] = df_test['APR Medical Surgical Description'].map(medical_surgical_mapping)
df_test['Emergency Department Indicator'] = df_test['Emergency Department Indicator'].map(emergency_mapping)

df_test_encoded = pd.get_dummies(df_test, columns=['Patient Disposition', 'Payment Typology 1'])

# Align test data with training data
df_test_encoded = df_test_encoded.reindex(columns=X_train.columns, fill_value=0)

# Train the model
model = Ridge()
model.fit(X_train, y_train)

# Generate predictions
Y_pred = model.predict(df_test_encoded)

# Compute RMSE based on the provided logic
errors = (y_train - Y_pred) ** 2  # Computing the error vector
errors = np.sort(errors)
percentile_value = 90  # Top 90% of errors
elements = int(percentile_value * len(errors) / 100)  # Taking the best 90% errors
errors = errors[:elements]
average_square_error = np.mean(errors)  # Average of square errors
rmse = (average_square_error) ** 0.5  # Root mean square

# Output RMSE
print("Objective Function obtained on the test set = " + str(rmse))

KeyError: 'Hospital_Zip'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/COL774_AS1/train.csv')

# Ensure the 'Total Costs' column is numeric
df['Total Costs'] = pd.to_numeric(df['Total Costs'], errors='coerce')

# Create the 'Hospital_Zip' feature
df['Hospital_Zip'] = df['Facility Name'].astype(str) + '_' + df['Zip Code - 3 digits'].astype(str)

def target_encode(train, target, col, alpha=1):
    mean_target = train[target].mean()  # Calculate the global mean of the target
    counts = train.groupby(col)[target].count()
    means = train.groupby(col)[target].mean()

    # Apply smoothing
    smoothed_means = (counts * means + alpha * mean_target) / (counts + alpha)

    return train[col].map(smoothed_means)

# Apply target encoding
df['Hospital_Zip_encoded'] = target_encode(df, 'Total Costs', 'Hospital_Zip')
df = df.drop(columns=['Hospital_Zip', 'Hospital Service Area', 'Hospital County', 'Operating Certificate Number', 'Permanent Facility Id', 'Facility Name', 'Zip Code - 3 digits', 'Payment Typology 2',
                      'Payment Typology 3', 'Birth Weight', 'CCSR Diagnosis Description', 'CCSR Procedure Description', 'APR DRG Description', 'APR MDC Description', 'APR Severity of Illness Code',
                      'CCSR Diagnosis Code','CCSR Procedure Code','APR DRG Code'])
df['Age Group'] = target_encode(df, 'Total Costs', 'Age Group')
df['Gender'] = target_encode(df, 'Total Costs', 'Gender')
df['Race'] = target_encode(df, 'Total Costs', 'Race')
df['Ethnicity'] = target_encode(df, 'Total Costs', 'Ethnicity')

#df['CCSR Diagnosis Code'] = target_encode(df, 'Total Costs', 'CCSR Diagnosis Code')
#df['CCSR Procedure Code'] = target_encode(df, 'Total Costs', 'CCSR Procedure Code')
#df['APR DRG Code'] = target_encode(df, 'Total Costs', 'APR DRG Code')
#df['APR MDC Code'] = target_encode(df, 'Total Costs', 'APR MDC Code')

# Define new label encoding mappings
severity_mapping = {3: 1, 4:2, 2:3, 1:4}
mortality_mapping = {3: 1, 4:2, 2:3, 1:4}
medical_surgical_mapping = {2: 0, 1: 1, 3: 2}
emergency_mapping = {1: 0, 2: 1}

df['APR Severity of Illness Description'] = df['APR Severity of Illness Description'].map(severity_mapping)
df['APR Risk of Mortality'] = df['APR Risk of Mortality'].map(mortality_mapping)
df['APR Medical Surgical Description'] = df['APR Medical Surgical Description'].map(medical_surgical_mapping)
df['Emergency Department Indicator'] = df['Emergency Department Indicator'].map(emergency_mapping)

df_encoded = pd.get_dummies(df, columns=['Patient Disposition', 'Payment Typology 1','APR MDC Code'])

# Define features (X) and target variable (y)
X = df_encoded.drop(columns=['Total Costs'])
y = df_encoded['Total Costs']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = Ridge()
model.fit(X_train, y_train)

# Generate predictions
Y_pred = model.predict(X_test)

# Compute RMSE based on the provided logic
errors = (y_test - Y_pred) ** 2  # Computing the error vector
errors = np.sort(errors)
percentile_value = 90  # Top 90% of errors
elements = int(percentile_value * len(errors) / 100)  # Taking the best 90% errors
errors = errors[:elements]
average_square_error = np.mean(errors)  # Average of square errors
rmse = (average_square_error) ** 0.5  # Root mean square

# Output RMSE
print("Objective Function obtained on the test set = " + str(rmse))


Objective Function obtained on the test set = 15991.139688901061


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/COL774_AS1/train.csv')

# Ensure 'Total Costs' column is numeric and handle missing values
df['Total Costs'] = pd.to_numeric(df['Total Costs'], errors='coerce')
df = df.dropna(subset=['Total Costs'])  # Drop rows where 'Total Costs' is NaN

# Create interaction features
df['Hospital_Zip'] = df['Facility Name'].astype(str) + '_' + df['Zip Code - 3 digits'].astype(str)
df['Age_Gender'] = df['Age Group'].astype(str) + '_' + df['Gender'].astype(str)
df['Race_Ethnicity'] = df['Race'].astype(str) + '_' + df['Ethnicity'].astype(str)

def target_encode(train, target, col, alpha=1):
    mean_target = train[target].mean()  # Calculate the global mean of the target
    counts = train.groupby(col)[target].count()
    means = train.groupby(col)[target].mean()

    # Apply smoothing
    smoothed_means = (counts * means + alpha * mean_target) / (counts + alpha)

    return train[col].map(smoothed_means)

# Apply target encoding
df['Hospital_Zip_encoded'] = target_encode(df, 'Total Costs', 'Hospital_Zip')
df['Age_Gender_encoded'] = target_encode(df, 'Total Costs', 'Age_Gender')
df['Race_Ethnicity_encoded'] = target_encode(df, 'Total Costs', 'Race_Ethnicity')

df = df.drop(columns=['Age_Gender', 'Race_Ethnicity', 'Age Group', 'Hospital_Zip', 'Gender',
                      'Facility Name', 'Zip Code - 3 digits'])

# Apply target encoding to additional columns
encoding_columns = ['CCSR Diagnosis Code', 'CCSR Procedure Code', 'APR DRG Code', 'APR MDC Code',
                     'APR Severity of Illness Code', 'Patient Disposition', 'Payment Typology 2',
                     'Payment Typology 3', 'Birth Weight', 'CCSR Diagnosis Description',
                     'CCSR Procedure Description', 'APR DRG Description', 'APR MDC Description','Emergency Department Indicator',
                    'APR Medical Surgical Description','APR Risk of Mortality','APR Severity of Illness Description']

one_hot_columns = []
target_columns  = []
label_columns   = []
for col in encoding_columns:
    df[col] = target_encode(df, 'Total Costs', col)

# Define new label encoding mappings
mappings = {
}

for col, mapping in mappings.items():
    df[col] = df[col].map(mapping)

# Use one-hot encoding for categorical variables
df_encoded = pd.get_dummies(df, columns=['Payment Typology 1'])

# Define features (X) and target variable (y)
X = df_encoded.drop(columns=['Total Costs'])
y = df_encoded['Total Costs']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
model = Ridge()
model.fit(X_train_scaled, y_train)

# Generate predictions
Y_pred = model.predict(X_test_scaled)

# Compute RMSE based on the provided logic
errors = (y_test - Y_pred) ** 2  # Computing the error vector
errors = np.sort(errors)
percentile_value = 90  # Top 90% of errors
elements = int(percentile_value * len(errors) / 100)  # Taking the best 90% errors
errors = errors[:elements]
average_square_error = np.mean(errors)  # Average of square errors
rmse = (average_square_error) ** 0.5  # Root mean square

# Output RMSE
print("Objective Function obtained on the test set = " + str(rmse))




Objective Function obtained on the test set = 12882.591294246049


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/COL774_AS1/train.csv')

# Ensure 'Total Costs' column is numeric and handle missing values
df['Total Costs'] = pd.to_numeric(df['Total Costs'], errors='coerce')
df = df.dropna(subset=['Total Costs'])  # Drop rows where 'Total Costs' is NaN

# Create the 'Hospital_Zip' feature
df['Hospital_Zip'] = df['Facility Name'].astype(str) + '_' + df['Zip Code - 3 digits'].astype(str)
df['Age_Gender'] = df['Age Group'].astype(str) + '_' + df['Gender'].astype(str)
df['Race_Ethnicity'] = df['Race'].astype(str) + '_' + df['Ethnicity'].astype(str)

def target_encode(train, target, col, alpha=1):
    mean_target = train[target].mean()  # Calculate the global mean of the target
    counts = train.groupby(col)[target].count()
    means = train.groupby(col)[target].mean()

    # Apply smoothing
    smoothed_means = (counts * means + alpha * mean_target) / (counts + alpha)

    return train[col].map(smoothed_means)

# Apply target encoding
df['Hospital_Zip_encoded'] = target_encode(df, 'Total Costs', 'Hospital_Zip')
df['Age_Gender_encoded'] = target_encode(df, 'Total Costs', 'Age_Gender')
df['Race_Ethnicity_encoded'] = target_encode(df, 'Total Costs', 'Race_Ethnicity')

df = df.drop(columns=['Age_Gender','Race_Ethnicity','Age Group','Hospital_Zip','Gender','Hospital Service Area', 'Hospital County', 'Operating Certificate Number', 'Permanent Facility Id', 'Facility Name', 'Zip Code - 3 digits'
                      ])

# Apply target encoding and label encoding
encoding_columns = ['CCSR Diagnosis Code', 'CCSR Procedure Code', 'APR DRG Code','APR MDC Code','APR Severity of Illness Code','Patient Disposition','Payment Typology 2','Payment Typology 3', 'Birth Weight', 'CCSR Diagnosis Description', 'CCSR Procedure Description', 'APR DRG Description', 'APR MDC Description']
for col in encoding_columns:
    df[col] = target_encode(df, 'Total Costs', col)

# Define new label encoding mappings
mappings = {
    'APR Severity of Illness Description': {3: 1, 4: 2, 2: 3, 1: 4},
    'APR Risk of Mortality': {3: 1, 4: 2, 2: 3, 1: 4},
    'APR Medical Surgical Description': {2: 1, 1: 3, 3: 5},
    'Emergency Department Indicator': {1: 1, 2: 4}
}

for col, mapping in mappings.items():
    df[col] = df[col].map(mapping)

df_encoded = pd.get_dummies(df, columns=['Payment Typology 1'])

# Define features (X) and target variable (y)
X = df_encoded.drop(columns=['Total Costs'])
#print(X.columns)
y = df_encoded['Total Costs']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



Objective Function obtained on the test set = 12846.032179378994


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Load the datasets
df_train = pd.read_csv('/content/drive/MyDrive/COL774_AS1/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/COL774_AS1/test.csv')

# Define target encoding function
def target_encode(train, target, col, alpha=1):
    mean_target = train[target].mean()  # Calculate the global mean of the target
    counts = train.groupby(col)[target].count()
    means = train.groupby(col)[target].mean()

    # Apply smoothing
    smoothed_means = (counts * means + alpha * mean_target) / (counts + alpha)

    return train[col].map(smoothed_means)

# Create a dictionary for target encoding during training
def create_encoding_dict(train, target_col, trgt_cols, alpha=1):
    encoding_dict = {}
    mean_target = train[target_col].mean()
    for col in trgt_cols:
        counts = train.groupby(col)[target_col].count()
        means = train.groupby(col)[target_col].mean()
        smoothed_means = (counts * means + alpha * mean_target) / (counts + alpha)
        encoding_dict[col] = smoothed_means
    return encoding_dict

# Define preprocessing function
def preprocess_data(df, encoding_dict=None, target_col=None):
    # Create new features
    df['Hospital_Zip'] = df['Facility Name'].astype(str) + '_' + df['Zip Code - 3 digits'].astype(str)
    df['Age_Gender'] = df['Age Group'].astype(str) + '_' + df['Gender'].astype(str)
    df['Race_Ethnicity'] = df['Race'].astype(str) + '_' + df['Ethnicity'].astype(str)

    # Drop unnecessary columns
    drop_cols = ['Hospital Service Area', 'Hospital County', 'Operating Certificate Number',
                 'Permanent Facility Id', 'Facility Name', 'Zip Code - 3 digits', 'Birth Weight',
                 'Age Group', 'Gender', 'Race', 'Ethnicity',
                 'Payment Typology 2', 'Payment Typology 3', 'CCSR Diagnosis Description',
                 'CCSR Procedure Description', 'APR DRG Description', 'APR MDC Description']

    df = df.drop(columns=[col for col in drop_cols if col in df.columns])

    # Apply target encoding if target column is provided
    trgt_cols = [
        'Hospital_Zip', 'Age_Gender', 'Race_Ethnicity', 'CCSR Diagnosis Code', 'CCSR Procedure Code',
        'APR DRG Code', 'APR MDC Code', 'APR Severity of Illness Code', 'Patient Disposition',
        'Payment Typology 1'
    ]

    if target_col:
        for col in trgt_cols:
            if col in df.columns:
                df[col] = target_encode(df, target_col, col)
            else:
                print(f"Warning: Column {col} not found in training data.")
    else:
        # For test data, use encoding from the dictionary
        for col in trgt_cols:
            if col in df.columns:
                if col in encoding_dict:
                    df[col] = df[col].map(encoding_dict[col]).fillna(encoding_dict[col].mean())
                else:
                    df[col] = np.nan

    # Apply one-hot encoding
    ohot_cols = ['Type of Admission', 'APR Medical Surgical Description', 'Emergency Department Indicator']
    df_encoded = pd.get_dummies(df, columns=ohot_cols)

    return df_encoded

# Preprocess training data to create new features
df_train_preprocessed = preprocess_data(df_train, target_col='Total Costs')

# Create encoding dictionary from preprocessed training data
trgt_cols = [
    'Hospital_Zip', 'Age_Gender', 'Race_Ethnicity', 'CCSR Diagnosis Code', 'CCSR Procedure Code',
    'APR DRG Code', 'APR MDC Code', 'APR Severity of Illness Code', 'Patient Disposition',
    'Payment Typology 1'
]
encoding_dict = create_encoding_dict(df_train_preprocessed, 'Total Costs', trgt_cols)

# Preprocess training data again to apply encoding
df_train_encoded = preprocess_data(df_train, encoding_dict=encoding_dict, target_col='Total Costs')
X_train = df_train_encoded.drop(columns=['Total Costs'])
y_train = df_train_encoded['Total Costs']

# Preprocess test data
df_test_encoded = preprocess_data(df_test, encoding_dict=encoding_dict)  # No target column for test data

# Ensure the test data has the same columns as the training data
missing_cols = set(X_train.columns) - set(df_test_encoded.columns)
for col in missing_cols:
    df_test_encoded[col] = 0  # Add missing columns with default value of 0

df_test_encoded = df_test_encoded[X_train.columns]  # Reorder columns to match training set

# Standardize the features
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(df_test_encoded)

# Train the model
model = Ridge()
model.fit(X_train, y_train)

# Generate predictions
Y_pred = model.predict(df_test_encoded)

# Load the true values from test_pred.csv
df_test_pred = pd.read_csv('/content/drive/MyDrive/COL774_AS1/test_pred.csv')
y_true = df_test_pred['Total Costs'].to_numpy()

# Check dimensions
if Y_pred.shape[0] != y_true.shape[0]:
    print("Prediction file has wrong dimensions for part c")
else:
    # Compute errors
    errors = (y_true - Y_pred) ** 2
    errors = np.sort(errors)

    percentile_value = 90  # Percentage of samples to consider
    elements = int(percentile_value * len(errors) / 100)  # Taking the best 90% errors
    errors = errors[:elements]

    average_square_error = np.mean(errors)  # Average of square errors
    rmse = np.sqrt(average_square_error)  # Root Mean Square Error

    # Print RMSE
    print("Objective Function obtained on the test set =", rmse)


Objective Function obtained on the test set = 41456.47243582985


In [ ]:
X_train.to_csv("X_train.csv")
y_train.to_csv("y_train.csv")
df_test_encoded.to_csv("df_test_encoded.csv")

In [ ]:
import sys
import pandas as pd
import numpy as np
from numpy import linalg
import argparse

def print_usage():
    """Print the usage instructions for the script."""
    print("Usage: python linear.py <prob_type a or b> <train_file> <test_file> <sample_weights/regularization> <model_predictions> <model_weights> [<lambda_out>]")
    sys.exit(1)

def load_data(train_file_path, test_file_path, input_file_path):
    try:
        df_train = pd.read_csv(train_file_path)
        df_test = pd.read_csv(test_file_path)
        df_input = pd.read_csv(input_file_path, header=None, dtype=str)
        return df_train, df_test, df_input
    except Exception as e:
        print(f"Unexpected error: {e}")
        sys.exit(1)

def process_data(input_data, prob_type):
    try:
        if data is not None:
            train, test, input = data
            train_nparray = train.to_numpy()
            test_nparray = test.to_numpy()
            X0 = train_nparray[:, :-1]
            X = np.hstack([np.ones((X0.shape[0], 1)), X0 ])
            Y = train_nparray[:, -1].reshape(-1, 1)
            X_test = np.hstack([np.ones((test_nparray.shape[0], 1)), test_nparray ])

            if prob_type == 'a':
                input_variables = input.to_numpy().astype(np.float64).flatten()
                if len(input_variables) != X0.shape[0]:
                    raise ValueError("Number of weights must match the number of samples in X")
            else:
                input_variables = [float(element) for element in input[0].tolist()]

        return X, Y, input_variables, X_test
    except Exception as e:
        print(f"Unexpected error in processing: {e}")
        sys.exit(1)

def get_lambda(X, Y, lambda_values, num_folds = 10):
    fold_size = len(X) // num_folds
    indices = np.arange(len(X))
    print("Lambda values: ", lambda_values)
    print("Total indices: ",len(indices))

    best_lambda = None
    lowest_cv_error = float('inf')

    for lmbda in lambda_values:
        cv_errors = []
        for i in range(num_folds):
            #print(i*fold_size,(i+1)*fold_size, (i+1)*fold_size-i*fold_size)

            test_indices = indices[i*fold_size:(i+1)*fold_size]
            train_indices = np.delete(indices, test_indices)

            X_train, X_test = X[train_indices], X[test_indices]
            Y_train, Y_test = Y[train_indices], Y[test_indices]

            I = np.eye(X_train.shape[1])  # Identity matrix for regularization
            XTX = X_train.T @ X_train + lmbda * I
            XTY = X_train.T @ Y_train

            w = linalg.solve(XTX, XTY)

            Y_pred = X_test @ w
            mse = np.mean((Y_test - Y_pred) ** 2)
            cv_errors.append(mse)

        #this is for avg, if sum you want, you use np.sum
        total_cv_error = np.sum(cv_errors)

        if total_cv_error < lowest_cv_error:
            lowest_cv_error = total_cv_error
            best_lambda = lmbda

    return best_lambda

def generate_predictions(X_test, w_final, predictions_file, weights_file):
    w_final = w_final.reshape(-1, 1)
    Y_pred = X_test @ w_final
    np.savetxt(weights_file, w_final, delimiter=',', comments='')
    print(f"Weights saved to {weights_file}")
    np.savetxt(predictions_file, Y_pred, delimiter=',', comments='')
    print(f"Predictions saved to {predictions_file}")


def solve_a(processed_data, model_predictions_file, model_weights_file):
    X, Y, U, X_test = processed_data
    XTUX = X.T @ (X * U.reshape(-1, 1))
    XTUY = X.T @ (Y * U.reshape(-1, 1))
    weights = np.linalg.solve(XTUX, XTUY)
    generate_predictions(X_test, weights, model_predictions_file, model_weights_file)

def solve_b(processed_data, model_predictions_file, model_weights_file, best_lambda_file):
    X, Y, lembdas, X_test = processed_data
    best_lambda = get_lambda(X, Y, lembdas, num_folds=10)
    np.savetxt(best_lambda_file, [best_lambda], delimiter=',', comments='')
    print(f"Best Lambda saved to {best_lambda_file}")

    # Train the final model with the best lambda on the entire dataset
    I = np.eye(X.shape[1])
    XTX_regularised = X.T @ X + best_lambda * I
    XTY = X.T @ Y
    w_final = linalg.solve(XTX_regularised, XTY)
    generate_predictions(X_test, w_final, model_predictions_file, model_weights_file)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Process command-line arguments for the script.")
    parser.add_argument('prob_type', type=str, choices=["a", "b"], help="Problem type")
    parser.add_argument('train_file', type=str, help='Path to the training file.')
    parser.add_argument('test_file', type=str, help='Path to the test file.')
    parser.add_argument('input_file_1', type=str, help='Path to the either weights or regularization data')
    parser.add_argument('prediction_file', type=str, help='Path to the prediction file.')
    parser.add_argument('model_out', type=str, help='Path to the model output file.')
    parser.add_argument('lambda_out', type=str, nargs='?', default=None, help='Path to the lambda output file (optional).')

    args = parser.parse_args()

    data = load_data(args.train_file, args.test_file, args.input_file_1)
    processed_data = process_data(data, args.prob_type)
    if args.prob_type == 'b':
        if  args.lambda_out is None:
            print("Error: For problem type 'b', <lambda_out> is required.")
            print_usage()
        solve_b(processed_data, args.prediction_file, args.model_out, args.lambda_out)
    elif args.prob_type == 'a':
        solve_a(processed_data, args.prediction_file, args.model_out)
    else:
        print("Error: Incorrect arguments.")
        print_usage()


Objective Function obtained on the test set = 24490.48552848473


In [ ]:
out_inp = sys.argv[1]
weight_inp = sys.argv[2]
out_model = sys.argv[3]
weight_model = sys.argv[4]



Train_file_path = 'train.csv'
Lamda_file_path = 'regularization.txt'
Weights1_file_path = 'sample_weights1.txt'
Weights2_file_path = 'sample_weights2.txt'

#Read Training file, weights file
df_train  = pd.read_csv(Train_file_path)
df_W1     = pd.read_csv(Weights1_file_path, delimiter='\t', header=None, dtype=str)
df_W2     = pd.read_csv(Weights2_file_path, delimiter='\t', header=None, dtype=str)
df_lambda = pd.read_csv(Lamda_file_path, header=None)


#Get Numpy array, X0 without bias values, X with ones for bias, X^T and Y from the File
numpy_array = df_train.to_numpy()
X0 = numpy_array[:, :-1]
X = np.hstack([np.ones((X0.shape[0], 1)), X0 ])
X_T = X.T
Y = numpy_array[:, -1].reshape(-1, 1)


U = df_W1.to_numpy().astype(np.float64).flatten()
print("Shapes: X0, U: ", X0.shape[0], len(U))
if len(U) != X0.shape[0]:
    raise ValueError("Number of weights must match the number of samples in X")

XTUX = np.zeros((X.shape[1], X.shape[1]))
XTUY = np.zeros((X.shape[1], 1))

for i in range(X.shape[0]):
    ui = U[i]
    x_i = X[i, :].reshape(-1, 1)
    XTUX += ui * (x_i @ x_i.T)
    XTUY += ui * x_i @ Y[i, :].reshape(-1, 1)

weights = np.linalg.solve(XTUX, XTUY)

#print(weights)